In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.model_selection import train_test_split 
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv("D:\\Sandesh\\100-days-of-machine-learning\\day29-sklearn-pipelines\\train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.columns
df=df[['PassengerId','Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked','Survived']]
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [5]:
df.drop(columns=['PassengerId',"Name",'Ticket','Cabin'],inplace=True)
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,male,22.0,1,0,7.2500,S,0
1,1,female,38.0,1,0,71.2833,C,1
2,3,female,26.0,0,0,7.9250,S,1
3,1,female,35.0,1,0,53.1000,S,1
4,3,male,35.0,0,0,8.0500,S,0


In [10]:
df.isnull().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
Survived      0
dtype: int64

In [6]:
X=df.iloc[:,0:7]
Y=df.iloc[:,-1]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=1)

In [23]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2

from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn.tree import DecisionTreeClassifier


### Pipeline Creation

In [9]:

# First column tranformer for age and embarked features.
# To Fill the null values.

trf1= ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
    ],remainder='passthrough')


In [12]:

# Second column transformer for sex and embarked features.
# To convert category into numeric values.

trf2= ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
    ],remainder='passthrough')


In [25]:

# 3 column transformer for scaling the all feature values.

trf3=ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])


In [31]:
# 4  - Feature Selection

trf4 = SelectKBest(score_func=chi2,k=8)

#5 - Decision Tree

trf5 = DecisionTreeClassifier()

### Create Pipeline 

In [32]:
pipe= Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5),
])

In [33]:

# To check the graph of the pipeline
from sklearn import set_config
set_config(display='diagram')

pipe.fit(X_train,y_train)

# If we are using the algorithm then go with fit only.
# If we are not using the algo such as decision tree at the end then we will go 
# with fit_transform option

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x00000227B99511F0>)),
                ('trf5', DecisionTreeClassifier())])

### We can check the values for every pipeline available.





In [36]:

# For imputation option

pipe.named_steps['trf1'].transformers_[0][1].statistics_


array([30.16623239])

In [34]:
from sklearn.metrics import accuracy_score

y_pred=pipe.predict(X_test)
accuracy_score(y_test,y_pred)

0.6201117318435754

### Cross Validation using Pipeline 

In [37]:
from sklearn.model_selection import cross_val_score

cross_val_score(pipe,X_train,y_train,cv=5,scoring='accuracy').mean()

0.6404510981975771

### GridSearch using Pipeline 

In [38]:

params={
    'trf5__max_depth':[1,2,3,4,5,None]
}


In [39]:
from sklearn.model_selection import GridSearchCV

grid=GridSearchCV(pipe,params,cv=5,scoring='accuracy')
grid.fit(X_train,y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x00000227B99511F0>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [40]:
grid.best_score_

0.6404510981975771

In [41]:
grid.best_params_

{'trf5__max_depth': 4}

* After dumping the pickle file , whenever we load the pickle file , we just have to pass the i/p then we dont have to make any changes out there , even if we make changes in pipeline and create new pipeline, after loading the pickle file , at the deployment we just have to pass the input, if we dont use the pipeline , then if we make any changes in implementation part , we have to do the same changes part in deployment stage as well.

* That is the advantage of using pipeline it gets easy on the deployment part.
